# Parte 1: Dados - Users_Followers

In [60]:
import pandas as pd
import re
import numpy as np

def fs_to_dataframe(fs):
    return pd.DataFrame({'User': re.split('\W+', fs)})

def create_edges_df(batches):
    return pd.concat(batches,
        ignore_index=True)

In [61]:
# load dataset

gh_users_allFollowers = pd.read_csv('temp_users_all-followers.csv')
gh_users_allFollowers.head()

,User,All_Followers,nF
0,tarruda,"Sannis,danielmahon,csjaba,FergusRedican,Victor...",570
1,mairatma,"brunocoelho,henvic,eduardolundgren,aperrelli,a...",363
2,joselitojunior1,"renatooliveira,jeffesonmaia,jotaefe,duartefq,J...",350
3,marcelcaraciolo,"thiagoarrais,brunojm,henriquebastos,macndesign...",330
4,luanfonceca,"brunohenrique,luizvarela,gladson,lucasbibiano,...",301


In [62]:
# normalizing

gh_users_allFollowers['All_Followers'] = gh_users_allFollowers['All_Followers'].fillna('')
gh_users_allFollowers['All_Followers'] = gh_users_allFollowers['All_Followers'].apply(fs_to_dataframe) # apenas para simplificar operacoes 

gh_users_allFollowers.head()

,User,All_Followers,nF
0,tarruda,User 0 S...,570
1,mairatma,User 0 brunocoelho ...,363
2,joselitojunior1,User 0 renatooliveira 1 ...,350
3,marcelcaraciolo,User 0 thiagoarrais 1 ...,330
4,luanfonceca,User 0 brunohenrique 1 ...,301


In [63]:
# Dataframe incluindo colunas contendo apenas seguidores recifenses

gh_users_Followers = gh_users_allFollowers.copy()

gh_users_Followers['Followers'] = gh_users_Followers['All_Followers'].apply(pd.DataFrame({'User':gh_users_Followers['User']}).merge)
gh_users_Followers['nFs'] = gh_users_Followers['Followers'].apply(len)

gh_users_Followers.head()

,User,All_Followers,nF,Followers,nFs
0,tarruda,User 0 S...,570,User 0 henriquemenezes 1 paul...,5
1,mairatma,User 0 brunocoelho ...,363,User 0 simoneas02 1 ...,19
2,joselitojunior1,User 0 renatooliveira 1 ...,350,User 0 luanfonceca 1 rena...,23
3,marcelcaraciolo,User 0 thiagoarrais 1 ...,330,User 0 luanfonceca 1 ...,28
4,luanfonceca,User 0 brunohenrique 1 ...,301,User 0 deividazevedo2 1 ...,9


In [64]:
# foco na visualização dos Followers recifenses 
# (mantendo no DataFrame todos os seguidores(recifenses e não recifenses))

gh_users_Followers = gh_users_Followers.loc[:, ['User', 'Followers', 'nFs', 'All_Followers', 'nF']]
gh_users_Followers.sort_values('nFs', axis=0, ascending=False, inplace=True)

gh_users_Followers.head()

,User,Followers,nFs,All_Followers,nF
12,fernandocastor,User 0 joselitojunior1 1 ...,48,User 0 guiocavalcanti ...,129
8,filipeximenes,User 0 luanfonceca ...,40,User 0 luisgabr...,177
7,simoneas02,User 0 talitaolive...,37,User 0 ElsonBarcelos ...,250
9,renatooliveira,User 0 joselitojunior1 1 ...,35,User 0 adrianomelo 1 ...,166
26,lailsonbm,User 0 luanfonceca 1 ...,29,User 0 kraudio 1 ...,73


In [65]:
# todos os usuarios recifenses (nem todos serao considerados como nos do graph - ver Parte 2) 
nodes_preview = gh_users_Followers.loc[:,['User','nFs', 'nF']]
nodes_preview.columns = ['Id', 'In-Degree', 'Full-In-Degree']

nodes_preview.head()

,Id,In-Degree,Full-In-Degree
12,fernandocastor,48,129
8,filipeximenes,40,177
7,simoneas02,37,250
9,renatooliveira,35,166
26,lailsonbm,29,73


In [66]:
# extração das edges

pairs = zip(gh_users_Followers.User, gh_users_Followers.Followers)

edges = [pd.DataFrame([{'Source':str(f), 'Target':str(u)} for f in fs['User']]) for (u,fs) in pairs]

edges_df = create_edges_df(edges)
edges_df.head()

,Source,Target
0,joselitojunior1,fernandocastor
1,filipeximenes,fernandocastor
2,renatooliveira,fernandocastor
3,henvic,fernandocastor
4,fjsj,fernandocastor


In [67]:
# formatando

gh_users_Followers['Followers'] = gh_users_Followers['Followers'].apply(lambda x: x.apply(','.join))
gh_users_Followers['All_Followers'] = gh_users_Followers['All_Followers'].apply(lambda x: x.apply(','.join))

gh_users_Followers.head()

,User,Followers,nFs,All_Followers,nF
12,fernandocastor,"joselitojunior1,filipeximenes,renatooliveira,h...",48,"guiocavalcanti,adrianomelo,tacsio,x8lucas8x,fj...",129
8,filipeximenes,"luanfonceca,gileno,renatooliveira,luiztiago,fe...",40,"luisgabriel,renatooliveira,marciobarbosa,anton...",177
7,simoneas02,"talitaoliveira,ktquez,pcstl,karlafalcao,kessia...",37,"ElsonBarcelos,IsabelaDePaula,sergiockd,douglas...",250
9,renatooliveira,"joselitojunior1,luanfonceca,gileno,filipeximen...",35,"adrianomelo,x8lucas8x,luisgabriel,joselitojuni...",166
26,lailsonbm,"luanfonceca,gileno,filipeximenes,renatooliveir...",29,"kraudio,lucasmncastro,chillicoder,lmarinho,gvc...",73


In [68]:
# gh_users_Followers.to_csv('gh_users_Followers.csv', index=False)
# edges_df.to_csv('edges.csv', index=False)


# Parte 2: Análise dos Dados - Users_Followers

In [69]:
# numero de usuarios recifenses 
gh_users_Followers['User'].count()

1992

In [70]:
# numero de usuarios recifenses com seguidores recifenses
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nFs'] != 0]]
len(users_0_fs)

692

In [71]:
# numero de usuarios recifenses sem seguidores recifenses 
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nFs'] == 0]]
len(users_0_fs)

1300

In [72]:
# numero de usuarios recifenses com seguidores (recifenses ou nao) 
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nF'] != 0]]
len(users_0_fs)

1103

In [73]:
# # numero de usuarios recifenses sem seguidores (recifenses ou nao) 
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nF'] == 0]]
len(users_0_fs)

889

**- Número de nós considerados**

Note que como estes nós pertencem a uma aresta, eles representam os usuários que tem seguidores e/ou seguem alguém.

In [74]:
# extração dos nodes

# (usuarios recifenses, excluidos os que não tem seguidores se e somente se não seguem ninguem)
len(pd.unique(edges_df[['Source','Target']].values.ravel()))

828

In [75]:
# numero de arestas (somatorio do total de seguidores por usuario)
gh_users_Followers.nFs.sum()

2320

In [76]:
# revendo o numero de arestas
edges = pd.read_csv('edges.csv')
edges.count()

source    2320
target    2320
dtype: int64

In [77]:
# distribuicao do Grau
gh_users_Followers.nFs.value_counts()

0     1300
1      310
2      142
3       67
4       42
5       27
7       19
6       18
8       14
9       14
11       6
14       5
12       3
23       3
20       2
13       2
18       2
10       2
15       2
24       2
25       2
29       1
37       1
35       1
16       1
28       1
19       1
40       1
48       1
Name: nFs, dtype: int64

In [78]:
# distribuicao do grau - usuarios por ocorrencias da quantidade de seguidores 
groupby_nfs = gh_users_Followers.groupby('nFs')
groupby_nfs['User'].apply( ','.join)

nFs
0     eliene-mb,anniewalker,lucasvsr,nataliaalves,th...
1     filipe-torres,ericksantana,Kassio-Ferreira,koo...
2     sudorafa,gilesv,andreneto,marcoshmendes,tonnyv...
3     luhanlacerda,walber,mauLeal,tcostam,bebetocf,v...
4     vmms16,wwcoderecife,andreldm,lhew,brunovpl,cit...
5     favasconcelos,alinedoleron,djalmaafilho,Master...
6     vanessa,gcaraciolo,luizlago,omailson,rvlb-19,L...
7     pedroqueiroz,t0rr3sp3dr0,leticiamachado,felipe...
8     iagobelo,dvro,nielsonsantana,tomersimis,JoaoGF...
9     patrickrbc,luanfonceca,jeftarmascarenhas,ovict...
10                              miguelarauj1o,Cisneiros
11    lmarinho,leopoldomt,karlafalcao,irgmedeiros,br...
12           brunnogomes,jordanamorais,marcellustavares
13                            brunofarache,alexpessoajr
14    vinicius3w,thiagodiniz,diegonvs,victorlaerte,a...
15                                interaminense,dakerfp
16                                           pauloborba
18                                        gv

In [79]:
# distribuicao do grau - usuarios por ocorrencias da quantidade de seguidores 
groupby_nfs = gh_users_Followers.groupby('nFs')
groupby_nfs['User'].apply( ','.join)

nFs
0     eliene-mb,anniewalker,lucasvsr,nataliaalves,th...
1     filipe-torres,ericksantana,Kassio-Ferreira,koo...
2     sudorafa,gilesv,andreneto,marcoshmendes,tonnyv...
3     luhanlacerda,walber,mauLeal,tcostam,bebetocf,v...
4     vmms16,wwcoderecife,andreldm,lhew,brunovpl,cit...
5     favasconcelos,alinedoleron,djalmaafilho,Master...
6     vanessa,gcaraciolo,luizlago,omailson,rvlb-19,L...
7     pedroqueiroz,t0rr3sp3dr0,leticiamachado,felipe...
8     iagobelo,dvro,nielsonsantana,tomersimis,JoaoGF...
9     patrickrbc,luanfonceca,jeftarmascarenhas,ovict...
10                              miguelarauj1o,Cisneiros
11    lmarinho,leopoldomt,karlafalcao,irgmedeiros,br...
12           brunnogomes,jordanamorais,marcellustavares
13                            brunofarache,alexpessoajr
14    vinicius3w,thiagodiniz,diegonvs,victorlaerte,a...
15                                interaminense,dakerfp
16                                           pauloborba
18                                        gv

# Parte 3: Merge dos Dados

### - Métricas obtidas no Gephi

In [93]:
node_metrics = pd.read_csv('gephi/node_metrics.csv')

# nodes_data.columns = ['User', 'In_Degree', 'Out_Degree', 'Degree', 'Modularity_Class', 'Page_Rank', 'Eccentricity_Centrality', 'Closness_Centrality', 'Betweeness_Centrality', 'EigenVector_Centrality']
# nodes_data.sort_values('In_Degree', axis=0, ascending=False, inplace=True)
# nodes_data.to_csv('gephi/node_metrics.csv', index=False)

node_metrics.head()

,User,In_Degree,Out_Degree,Degree,Modularity_Class,Page_Rank,Eccentricity_Centrality,Closness_Centrality,Betweeness_Centrality,EigenVector_Centrality
0,fernandocastor,48,7,55,6,0.012520,8,0.253469,37787.361925,0.751562
1,filipeximenes,40,32,72,0,0.014185,9,0.267709,40297.561868,1.000000
2,simoneas02,37,26,63,9,0.011433,10,0.268627,40894.607104,0.211671
3,renatooliveira,35,28,63,0,0.010550,9,0.258613,26763.575801,0.994672
4,lailsonbm,29,1,30,0,0.013151,11,0.149686,1142.658235,0.838433


In [94]:
node_metrics.describe()

,In_Degree,Out_Degree,Degree,Modularity_Class,Page_Rank,Eccentricity_Centrality,Closness_Centrality,Betweeness_Centrality,EigenVector_Centrality
count,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000
mean,2.793478,2.793478,5.586957,7.602657,0.001208,6.553140,0.256711,1419.676329,0.042428
std,4.567072,4.667115,8.360292,6.902523,0.001698,4.938395,0.295489,4458.272707,0.107496
min,0.000000,0.000000,1.000000,-3.000000,0.000329,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.750000,0.000445,1.000000,0.127562,0.000000,0.001290
50%,1.000000,1.000000,3.000000,6.000000,0.000682,9.000000,0.191944,0.000000,0.006063
75%,3.000000,3.000000,6.000000,12.000000,0.001235,11.000000,0.224246,692.926692,0.029582
max,48.000000,46.000000,72.000000,33.000000,0.020541,14.000000,1.000000,40894.607104,1.000000


**Legenda Modularity_Class**. Adotei:  
- \-2 para as "comunidades" de tamanho 2
- \-3 para as "comunidades" de tamanho 3


In [91]:
gh_users_repositories = pd.read_csv('temp_users_repositories.csv')
temp_node_data = gh_users_Followers.merge(gh_users_repositories, left_on='User', right_on='User')

temp_node_data.head()

,User,Followers,nFs,All_Followers,nF,Repo_Member,mRepo_Language,Repo_Owner,oRepo_Language
0,fernandocastor,"joselitojunior1,filipeximenes,renatooliveira,h...",48,"guiocavalcanti,adrianomelo,tacsio,x8lucas8x,fj...",129,"curso-scm/scm,rafaelbrandao/deadlock-exception...","None,TeX,Java,Haskell","fernandocastor/5linguagens,fernandocastor/aula...","TeX,TeX,Java,Python,HTML,Haskell,Objective-C,O..."
1,filipeximenes,"luanfonceca,gileno,renatooliveira,luiztiago,fe...",40,"luisgabriel,renatooliveira,marciobarbosa,anton...",177,"tapanpandita/pocket,vintasoftware/django-apps-...","Python,HTML","filipeximenes/agendacultural,filipeximenes/bat...","HTML,JavaScript,Matlab,Python,Python,Python,Py..."
2,simoneas02,"talitaoliveira,ktquez,pcstl,karlafalcao,kessia...",37,"ElsonBarcelos,IsabelaDePaula,sergiockd,douglas...",250,"afonsopacifer/css-grid-layout-manual,CodeMigas...","None,None,JavaScript,Java,PHP","simoneas02/2017-goals,simoneas02/2018-goals,si...","None,None,None,None,Ruby,JavaScript,JavaScript..."
3,renatooliveira,"joselitojunior1,luanfonceca,gileno,filipeximen...",35,"adrianomelo,x8lucas8x,luisgabriel,joselitojuni...",166,"alessandroHenrique/coinpricemonitor,filipecmed...","Python,Python,JavaScript,JavaScript,JavaScript...","renatooliveira/authomatic,renatooliveira/celer...","Python,HTML,Python,Python,Python,JavaScript,Py..."
4,lailsonbm,"luanfonceca,gileno,filipeximenes,renatooliveir...",29,"kraudio,lucasmncastro,chillicoder,lmarinho,gvc...",73,"gvc/libertas,jeffp/enumerated_attribute,sergio...","Ruby,Ruby,CSS","lailsonbm/2d_semi_supervised,lailsonbm/ABNT2-L...","None,None,Ruby,Ruby,Ruby,Ruby,Matlab,Ruby,Java..."


In [95]:
node_data = temp_node_data.merge(node_metrics, left_on='User', right_on='User')
node_data.head()

,User,Followers,nFs,All_Followers,nF,Repo_Member,mRepo_Language,Repo_Owner,oRepo_Language,In_Degree,Out_Degree,Degree,Modularity_Class,Page_Rank,Eccentricity_Centrality,Closness_Centrality,Betweeness_Centrality,EigenVector_Centrality
0,fernandocastor,"joselitojunior1,filipeximenes,renatooliveira,h...",48,"guiocavalcanti,adrianomelo,tacsio,x8lucas8x,fj...",129,"curso-scm/scm,rafaelbrandao/deadlock-exception...","None,TeX,Java,Haskell","fernandocastor/5linguagens,fernandocastor/aula...","TeX,TeX,Java,Python,HTML,Haskell,Objective-C,O...",48,7,55,6,0.012520,8,0.253469,37787.361925,0.751562
1,filipeximenes,"luanfonceca,gileno,renatooliveira,luiztiago,fe...",40,"luisgabriel,renatooliveira,marciobarbosa,anton...",177,"tapanpandita/pocket,vintasoftware/django-apps-...","Python,HTML","filipeximenes/agendacultural,filipeximenes/bat...","HTML,JavaScript,Matlab,Python,Python,Python,Py...",40,32,72,0,0.014185,9,0.267709,40297.561868,1.000000
2,simoneas02,"talitaoliveira,ktquez,pcstl,karlafalcao,kessia...",37,"ElsonBarcelos,IsabelaDePaula,sergiockd,douglas...",250,"afonsopacifer/css-grid-layout-manual,CodeMigas...","None,None,JavaScript,Java,PHP","simoneas02/2017-goals,simoneas02/2018-goals,si...","None,None,None,None,Ruby,JavaScript,JavaScript...",37,26,63,9,0.011433,10,0.268627,40894.607104,0.211671
3,renatooliveira,"joselitojunior1,luanfonceca,gileno,filipeximen...",35,"adrianomelo,x8lucas8x,luisgabriel,joselitojuni...",166,"alessandroHenrique/coinpricemonitor,filipecmed...","Python,Python,JavaScript,JavaScript,JavaScript...","renatooliveira/authomatic,renatooliveira/celer...","Python,HTML,Python,Python,Python,JavaScript,Py...",35,28,63,0,0.010550,9,0.258613,26763.575801,0.994672
4,lailsonbm,"luanfonceca,gileno,filipeximenes,renatooliveir...",29,"kraudio,lucasmncastro,chillicoder,lmarinho,gvc...",73,"gvc/libertas,jeffp/enumerated_attribute,sergio...","Ruby,Ruby,CSS","lailsonbm/2d_semi_supervised,lailsonbm/ABNT2-L...","None,None,Ruby,Ruby,Ruby,Ruby,Matlab,Ruby,Java...",29,1,30,0,0.013151,11,0.149686,1142.658235,0.838433


In [96]:
# node_data.to_csv('gephi/node_data.csv', index=False) # depois disso tratei algumas repeticoes in loco